In [1]:
import argparse
import gym
import numpy as np
import torch
from sac import SAC
import robel
from updated_env import DKittyWalkRandom, DKittyOrientRandomDynamics

In [3]:
env = DKittyWalkRandom(device_path = 'd', torso_tracker_id=1, reset_type='scripted') #DKittyWalkRandom()#DKittyOrientRandomDynamics([2.]*8) #gym.make("DKittyOrientRandomDynamics-v0")
env.seed(42)
env.action_space.seed(42)
env._max_episode_steps = 160 ####### TO CHECK ITs consistent with PEARL (Arsen)
torch.manual_seed(42)
np.random.seed(42)
env.reset()

InitError_Init_HmdNotFoundPresenceFailed: 

In [6]:
env._device_path = 't'


In [9]:
env._torso_tracker_id = 1

In [11]:
env._torso_tracker_id

1

In [14]:
dir(env)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_action_space',
 '_add_component',
 '_close',
 '_components',
 '_configure_robot',
 '_configure_tracker',
 '_device_path',
 '_falling_reward',
 '_get_last_action',
 '_get_obs',
 '_get_total_reward',
 '_get_upright_obs',
 '_get_upright_rewards',
 '_gym_disable_underscore_compat',
 '_hardware_reset',
 '_heading_tracker_id',
 '_initial_heading_pos',
 '_initial_target_pos',
 '_initialize_action_space',
 '_initialize_observation_space',
 '_initialize_state_space',
 '_max_episode_steps',
 '_observation_keys',
 '_observation_space',
 '_preprocess_action',
 '_reset

In [3]:
env.render()

Creating window glfw


In [4]:
class Args:
    def __init__(self):
        self.policy = "Gaussian"
        self.eval = True
        self.gamma = 0.99
        self.tau = 0.005
        self.lr = 0.0003
        self.alpha = 0.2
        self.automatic_entropy_tuning = False
        self.batch_size = 256
        self.num_steps = 1000001
        self.hidden_size = 256
        self.update_per_step = 1
        self.start_steps = 10000
        self.target_update_interval = 1
        self.replay_size = 1000000
        self.cuda = True
args = Args()

In [5]:
agent = SAC(env.observation_space.shape[0], env.action_space, args)

In [6]:
agent.load_model("./models/sac_actor_HalfCheetah-v2_",

                 "./models/sac_critic_HalfCheetah-v2_")

Loading models from ./models/sac_actor_HalfCheetah-v2_ and ./models/sac_critic_HalfCheetah-v2_


In [7]:
from tqdm import trange

In [8]:
import numpy as np
np.cos(np.pi/180*25.)

0.9063077870366499

In [9]:
avg_reward = 0.
avg_success = 0.
episodes = 10
max_test_r = -np.inf
for _  in range(episodes):
    state = env.reset()
    env.render()
    episode_reward = 0
    episode_success = 0
    done = False
    for i in trange(80):
        action = agent.select_action(state, evaluate=True)
        next_state, reward, done, info = env.step(action)
        episode_reward += reward
        state = next_state
        if info['score/success']:
            episode_success = 1
    avg_success += episode_success       
    avg_reward += episode_reward
avg_reward /= episodes
avg_success /= episodes

print("----------------------------------------")
print("Test Episodes: {}, Avg. Success: {}".format(episodes, round(avg_success, 2)))
print("----------------------------------------")
print("Test Episodes: {}, Avg. Reward: {}".format(episodes, round(avg_reward, 2)))
print("----------------------------------------")
env.close()

100%|██████████| 80/80 [00:09<00:00,  8.75it/s]

----------------------------------------
Test Episodes: 10, Avg. Success: 0.0
----------------------------------------
Test Episodes: 10, Avg. Reward: -42213.43
----------------------------------------
